In [ ]:
import pandas as pd

#LÀM SẠCH DỮ LIỆU
# Đọc file
khach_hang = pd.read_csv('customer_info.csv')
hoa_don = pd.read_csv('invoice_header.csv')
chi_tiet = pd.read_csv('invoice_detail.csv')

# Làm sạch dữ liệu
khach_hang['dob'] = pd.to_datetime(khach_hang['dob'], errors='coerce')
khach_hang['full_name'] = khach_hang['full_name'].astype(str).str.strip().str.title()
khach_hang['full_name'] = khach_hang['full_name'].str.replace(r'[^\w\s]', '', regex=True)
khach_hang['region'] = khach_hang['region'].astype(str).str.strip().str.capitalize().replace('Nan', 'Không xác định')
khach_hang['customer_id'] = khach_hang['customer_id'].astype(str).str.strip()

hoa_don['date'] = pd.to_datetime(hoa_don['date'], errors='coerce',dayfirst=True)
hoa_don = hoa_don.dropna(subset=['date'])
hoa_don['customer_id'] = hoa_don['customer_id'].astype(str).str.strip()

chi_tiet['product_name'] = chi_tiet['product_name'].astype(str).str.strip()

chi_tiet['unit_price'] = chi_tiet['unit_price'].astype(str).str.replace(r'[^\d.]', '', regex=True)
chi_tiet['unit_price'] = pd.to_numeric(chi_tiet['unit_price'], errors='coerce').fillna(0)
chi_tiet['unit_price'] = chi_tiet['unit_price'].clip(lower=0)

chi_tiet['quantity'] = chi_tiet['quantity'].astype(str).str.replace(r'[^\d-]', '', regex=True)
chi_tiet['quantity'] = pd.to_numeric(chi_tiet['quantity'], errors='coerce').fillna(0).astype(int)
chi_tiet['quantity'] = chi_tiet['quantity'].clip(lower=0)

# Tính doanh thu
chi_tiet['doanh_thu'] = chi_tiet['quantity'] * chi_tiet['unit_price']

#MERGE TẠO FILE HOÀN CHỈNH
du_lieu_day_du = hoa_don.merge(khach_hang, on='customer_id', how='left')
du_lieu_day_du = du_lieu_day_du.merge(chi_tiet, on='invoice_id', how='left')

# Điền region thiếu
du_lieu_day_du['region'] = du_lieu_day_du['region'].fillna('Không xác định')

### Nhiệm vụ 5 – Pivot Table + Stack/Unstack

In [ ]:
# Pivot 1: Doanh thu theo khách hàng và sản phẩm
bang_tong_hop_khach_hang_san_pham = pd.pivot_table(
    du_lieu_day_du,
    values='doanh_thu',
    index='customer_id',
    columns='product_name',
    aggfunc='sum',
    fill_value=0
)
print("\nBảng tổng hợp doanh thu theo khách hàng và sản phẩm ")
display(bang_tong_hop_khach_hang_san_pham)

# Pivot 2: Doanh thu theo khu vực và sản phẩm
bang_tong_hop_khu_vuc_san_pham = pd.pivot_table(
    du_lieu_day_du,
    values='doanh_thu',
    index='region',
    columns='product_name',
    aggfunc='sum',
    fill_value=0
)
print("\nBảng tổng hợp doanh thu theo khu vực và sản phẩm")
display(bang_tong_hop_khu_vuc_san_pham)

# Stack:
du_lieu_dang_dai = bang_tong_hop_khu_vuc_san_pham.stack()
print("\nDữ liệu sau khi stack")
display(du_lieu_dang_dai.head(20))

# Unstack:
du_lieu_dang_rong_lai = du_lieu_dang_dai.unstack()
print("\nDữ liệu sau khi unstack")
display(du_lieu_dang_rong_lai)

# ==================== PHÂN TÍCH KẾT QUẢ ====================
san_pham_doanh_thu_cao_nhat = bang_tong_hop_khu_vuc_san_pham.sum().idxmax()
tong_doanh_thu_san_pham_cao_nhat = bang_tong_hop_khu_vuc_san_pham.sum().max()

khu_vuc_doanh_thu_cao_nhat = bang_tong_hop_khu_vuc_san_pham.sum(axis=1).idxmax()
tong_doanh_thu_khu_vuc_cao_nhat = bang_tong_hop_khu_vuc_san_pham.sum(axis=1).max()

san_pham_ban_chay_nhat_theo_khu_vuc = bang_tong_hop_khu_vuc_san_pham.idxmax(axis=1)

print("PHÂN TÍCH KẾT QUẢ")
print(f"Sản phẩm có doanh thu cao nhất : {san_pham_doanh_thu_cao_nhat}")
print(f"Tổng doanh thu: {tong_doanh_thu_san_pham_cao_nhat:,.0f} đồng")
print(f"Khu vực có tổng doanh thu cao nhất: {khu_vuc_doanh_thu_cao_nhat}")
print(f"Tổng doanh thu: {tong_doanh_thu_khu_vuc_cao_nhat:,.0f} đồng")
print("\nSản phẩm bán chạy nhất tại từng khu vực:")
print(san_pham_ban_chay_nhat_theo_khu_vuc)
print("Các ô có giá trị 0: Khu vực đó chưa có khách hàng nào mua sản phẩm")


Bảng tổng hợp doanh thu theo khách hàng và sản phẩm 


product_name,Ban mi,Dau an,Kem danh rang,Mi goi,Nuoc ngot,Sua tuoi
customer_id,,,,,,
C102,0.0,150000.0,0.0,0.0,0.0,135000.0
C103,192000.0,0.0,0.0,60000.0,0.0,0.0
C104,0.0,0.0,30000.0,0.0,0.0,18000.0
C105,0.0,24000.0,0.0,123000.0,24000.0,0.0
C107,0.0,12000.0,18000.0,40000.0,0.0,0.0
C109,0.0,0.0,45000.0,0.0,0.0,0.0
C111,0.0,0.0,0.0,30000.0,90000.0,0.0
C112,60000.0,0.0,0.0,60000.0,30000.0,90000.0
C113,0.0,0.0,0.0,0.0,0.0,0.0



Bảng tổng hợp doanh thu theo khu vực và sản phẩm


product_name,Ban mi,Dau an,Kem danh rang,Mi goi,Nuoc ngot,Sua tuoi
region,,,,,,
Bac,214000.0,114000.0,12000.0,318000.0,24000.0,84000.0
Bắc,192000.0,222000.0,54000.0,90000.0,304000.0,153000.0
Không xác định,164000.0,148000.0,126000.0,132000.0,0.0,40000.0
Nam,60000.0,12000.0,303000.0,136000.0,30000.0,150000.0
Trung,96000.0,120000.0,36000.0,75000.0,150000.0,180000.0



Dữ liệu sau khi stack


region          product_name 
Bac             Ban mi           214000.0
                Dau an           114000.0
                Kem danh rang     12000.0
                Mi goi           318000.0
                Nuoc ngot         24000.0
                Sua tuoi          84000.0
Bắc             Ban mi           192000.0
                Dau an           222000.0
                Kem danh rang     54000.0
                Mi goi            90000.0
                Nuoc ngot        304000.0
                Sua tuoi         153000.0
Không xác định  Ban mi           164000.0
                Dau an           148000.0
                Kem danh rang    126000.0
                Mi goi           132000.0
                Nuoc ngot             0.0
                Sua tuoi          40000.0
Nam             Ban mi            60000.0
                Dau an            12000.0
dtype: float64


Dữ liệu sau khi unstack


product_name,Ban mi,Dau an,Kem danh rang,Mi goi,Nuoc ngot,Sua tuoi
region,,,,,,
Bac,214000.0,114000.0,12000.0,318000.0,24000.0,84000.0
Bắc,192000.0,222000.0,54000.0,90000.0,304000.0,153000.0
Không xác định,164000.0,148000.0,126000.0,132000.0,0.0,40000.0
Nam,60000.0,12000.0,303000.0,136000.0,30000.0,150000.0
Trung,96000.0,120000.0,36000.0,75000.0,150000.0,180000.0


PHÂN TÍCH KẾT QUẢ
Sản phẩm có doanh thu cao nhất : Mi goi
Tổng doanh thu: 751,000 đồng
Khu vực có tổng doanh thu cao nhất: Bắc
Tổng doanh thu: 1,015,000 đồng

Sản phẩm bán chạy nhất tại từng khu vực:
region
Bac                      Mi goi
Bắc                   Nuoc ngot
Không xác định           Ban mi
Nam               Kem danh rang
Trung                  Sua tuoi
dtype: object
Các ô có giá trị 0: Khu vực đó chưa có khách hàng nào mua sản phẩm
